<a href="https://colab.research.google.com/github/studboii/ML-Colab_Demos/blob/main/Notebooks/MNIST_Neural_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 16.6MB/s 
     |████████████████████████████████| 1.1MB 49.0MB/s 
     |████████████████████████████████| 194kB 64.8MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
Mounted at /content/gdrive


In [ ]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
!mkdir Data-Sets

In [ ]:
from google.colab import files
uploader = files.upload()

Saving MNIST-As-PNG.tar.gz to MNIST-As-PNG.tar.gz


In [ ]:
!7z x "/content/MNIST-As-PNG.tar.gz" -so | 7z x -aoa -si -ttar -o"/content/Data-Sets/MNIST-As-PNG"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)


Extracting archive: 
--
Path = 
Type = tar
Code Page = UTF-8

  0%      0% 960 - mnist_png/testing/2/4135.png                                         0% 1928 - mnist_png/testing/7/746.png                                         0% 2853 - mnist_png/testing/9/5971.png                                          0% 3821 - mnist_png/testing/8/8699.png                                          0% 4788 - mnist_png/testing/5/720.png             

In [ ]:
path = Path("/content/Data-Sets/MNIST-As-PNG/mnist_png/")
Path.BASE_PATH = path

We save location of all the 0s and 1s in the training set

In [ ]:
Zeroes = (path/'training'/'0').ls().sorted()
Ones = (path/'training'/'1').ls().sorted()
Zeroes

(#5923) [Path('training/0/1.png'),Path('training/0/1000.png'),Path('training/0/10005.png'),Path('training/0/10010.png'),Path('training/0/10022.png'),Path('training/0/10025.png'),Path('training/0/10026.png'),Path('training/0/10045.png'),Path('training/0/10069.png'),Path('training/0/10071.png')...]

We open those adressed and create a tensor

In [ ]:
ZeroT = [tensor(Image.open(o)) for o in Zeroes]
OneT = [tensor(Image.open(o)) for o in Ones]

We then Stack all of our Tensors

In [ ]:
Stacked0 = torch.stack(ZeroT).float()/255
Stacked1 = torch.stack(OneT).float()/255

We concatenate the stacked tensors and create our train set with it. X is the data and Y holds the label

In [ ]:
train_x = torch.cat([Stacked0, Stacked1]).view(-1, 28*28)
train_y = tensor([1]*len(Zeroes) + [0]*len(Ones)).unsqueeze(1)
train_x.shape,train_y.shape
dset = list(zip(train_x,train_y))
dl = DataLoader(dset, batch_size=256)

Simialar to out train set we creat a validation batch repeating the steps we did for our training set

In [ ]:
ValidZero = (path/'testing'/'0').ls().sorted()
ValidOne = (path/'testing'/'1').ls().sorted()

We create our validation tensors

In [ ]:
ValidZeroT = [tensor(Image.open(o)) for o in ValidZero]
ValidOneT = [tensor(Image.open(o)) for o in ValidOne]

We stack them

In [ ]:
Val_Stack_Zero = torch.stack(ValidZeroT).float()/255
Val_One_Stack = torch.stack(ValidOneT).float()/255

We create our validation dataloader

In [ ]:
valid_x = torch.cat([Val_Stack_Zero, Val_One_Stack]).view(-1, 28*28)
valid_y = tensor([1]*len(ValidZeroT) + [0]*len(ValidOneT)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))
valid_dl = DataLoader(valid_dset,batch_size=256)

We define our loss function.

In [ ]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

We combine our dataloaders into a single Dataloader object

In [ ]:
dls = DataLoaders(dl, valid_dl)

We define a simple neural net for our data

In [ ]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

We define a function to calculate batch Accuracy

In [ ]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

We define our learner

In [ ]:
!mkdir MyModels

In [ ]:
learn = Learner(dls, simple_net, opt_func=SGD,loss_func=mnist_loss, metrics=batch_accuracy,path="/content/MyModels/")

We train our leaner and fit data.

In [ ]:
learn.fit(50,0.2)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.120295,0.197133,0.868558,00:00
1,0.044637,0.010693,0.996217,00:00
2,0.019952,0.005066,0.998582,00:00
3,0.010642,0.003563,0.998582,00:00
4,0.006749,0.002861,0.999054,00:00
5,0.004966,0.002451,0.999054,00:00
6,0.004058,0.002183,0.999054,00:00
7,0.003537,0.001993,0.999054,00:00
8,0.003200,0.001851,0.999054,00:00
9,0.002960,0.001742,0.999054,00:00


In [ ]:
learn.save("MyModel")

Path('/content/MyModels/models/MyModel.pth')